In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime as dt
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive')
import math
import numpy as np
from sklearn.model_selection import train_test_split

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Modified_data/new_Units.csv'
path2 ='/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Modified_data/new_Buildings.csv'
path3 = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/Data_transacations.csv'
path4 = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/DLD_Archive/Land_Registry.csv'
path5 = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/DLD_Archive/Rent_Contracts.csv'

In [ ]:
#tdf= pd.read_csv(path3)
bdf = pd.read_csv(path2)
udf = pd.read_csv(path)

<ipython-input-4-be00d17a4b65>:2: DtypeWarning: Columns (6,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  bdf = pd.read_csv(path2)
<ipython-input-4-be00d17a4b65>:3: DtypeWarning: Columns (6,9,11,21,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  udf = pd.read_csv(path)


In [ ]:
#ldf = pd.read_csv(path4)
tdf= pd.read_csv(path3)

In [ ]:
rdf = pd.read_csv(path5)

In [ ]:
removing_cols = [col for col in rdf.columns if col.endswith('_ar')]
removing_cols

['contract_reg_type_ar',
 'ejari_bus_property_type_ar',
 'ejari_property_type_ar',
 'ejari_property_sub_type_ar',
 'property_usage_ar',
 'project_name_ar',
 'master_project_ar',
 'area_name_ar',
 'nearest_landmark_ar',
 'nearest_metro_ar',
 'nearest_mall_ar',
 'tenant_type_ar']

In [ ]:
#rdf = rdf.drop(columns=removing_cols)

rdf.isnull().sum()

,0
contract_id,0
contract_reg_type_id,0
contract_reg_type_en,0
contract_start_date,0
contract_end_date,0
contract_amount,0
annual_amount,0
no_of_prop,0
line_number,0
is_free_hold,2973


In [ ]:
tdf['master_project_en'].nunique()
tdf[ 'project_name_en'].nunique()

2267

# For Removing

In [ ]:
removing_col = [col for col in df.columns if col.endswith('_ar')]
removing_col

[]

In [ ]:
Q3 =  df['meter_sale_price'].quantile(0.75)
Q1 = df['meter_sale_price'].quantile(0.25)
lower_bound = Q1 - 1.5 * (Q3-Q1)
upper_bound = Q3 + 1.5 * (Q3-Q1)
# Anything above upper bound and below lower bound are considered outliers.
# But for ease of use, We are dropping meter_sale_price < 1

In [ ]:
data_out = df[(df['meter_sale_price']  < 1) |  (df['meter_sale_price']  > upper_bound)]
data_out.to_excel("/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Modified_data/transactions_dropped_on meter_sale_price_original.xlsx")
data_out.shape
#4% of the data is outliers

(0, 40)

In [ ]:
#Data free from outliers
data_1 = df[(df['meter_sale_price']  >= 1) &  (df['meter_sale_price']  <= upper_bound)]
data_1.shape
#95% of 1.4M  = 1.36M records are there in Data_1

(1362082, 39)

In [ ]:
data_1['meter_sale_price'].groupby(by=data_1['property_type_en'], dropna=False).describe()

,count,mean,std,min,25%,50%,75%,max
property_type_en,,,,,,,,
Building,29351.0,8618.369561,8081.023151,1.00,1827.9800,5946.000,13614.2200,30342.11
Land,119762.0,7816.030064,6262.913285,1.00,2206.6000,6775.220,12054.8175,30355.71
Unit,943285.0,13140.957265,6279.563659,1.50,8244.1500,11840.270,17059.1100,30358.29
Villa,269684.0,7520.790813,4920.261046,1.44,3953.4075,6855.145,10177.5200,30358.23


In [ ]:
#data['instance_MonthYear'] = data['instance_Month'].astype('str') + '-' + data['instance_Year'].astype('str')


data_timeseries = data_1['meter_sale_price'].groupby(by=[data_1.instance_Year, data_1.instance_Month], dropna=False).describe()

data_timeseries.to_excel("C:/Users/umaho/OneDrive/Desktop/Flipose/Clean_out/data_timeseries.xlsx")

In [ ]:
## Adding time constraint -
#Data in the year 2000 to 2025

df2 = data_1[(data_1['instance_Year']  >= 2000) &  (data_1['instance_Year']  <= 2025)]
df2.shape

(1356407, 39)

In [ ]:
data_2.head()

,transaction_id,procedure_id,trans_group_id,trans_group_en,procedure_name_en,instance_date,property_type_id,property_type_en,property_sub_type_id,property_sub_type_en,...,no_of_parties_role_1,no_of_parties_role_2,no_of_parties_role_3,instance_Year,instance_Month,instance_Day,trans_X_1,trans_X_2,Trans_Year,trans_X_3
0,1-11-2024-10138,11,1,Sales,Sell,2024-03-19,4,Villa,NaN,NaN,...,8.0,1.0,0.0,2024.0,3.0,19.0,1,11,2024,10138
1,3-9-2002-39,9,3,Gifts,Grant,2002-03-25,1,Land,NaN,NaN,...,1.0,1.0,0.0,2002.0,3.0,25.0,3,9,2002,39
2,1-11-2016-12930,11,1,Sales,Sell,2016-11-02,4,Villa,NaN,NaN,...,1.0,1.0,0.0,2016.0,11.0,2.0,1,11,2016,12930
3,1-11-2005-300028,11,1,Sales,Sell,2005-02-28,4,Villa,NaN,NaN,...,1.0,1.0,0.0,2005.0,2.0,28.0,1,11,2005,300028
4,1-11-2010-17709,11,1,Sales,Sell,2010-12-09,2,Building,NaN,NaN,...,1.0,1.0,0.0,2010.0,12.0,9.0,1,11,2010,17709


In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
df_area_1 = df2[(df2['procedure_area']  > 1)]
#data_area_less_1.to_excel(data_out.to_csv("/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Modified_data/data_2_model.csv"))

In [ ]:
df_area_1.shape

(1356365, 39)

In [ ]:
df_area_1
Q3 =  df_area_1['procedure_area'].quantile(0.75)
Q1 = df_area_1['procedure_area'].quantile(0.25)
lower_bound = Q1 - 1.5 * (Q3-Q1)
upper_bound = Q3 + 1.5 * (Q3-Q1)
upper_bound

np.float64(469.235)

In [ ]:
df3 =df_area_1[(df_area_1['procedure_area'] <=upper_bound)]

In [ ]:
df3.shape

(1148886, 39)

In [ ]:
df3.to_csv("/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/Data_transacations.csv")

# New Section

In [ ]:
ar_pr = df['meter_sale_price'].groupby([df['area_name_en'],df['property_type_en']]).describe()

In [ ]:
ar_pr#.to_excel("/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Modified_data/Grouped_data_summary.xlsx")

count          mean           std      min  \
area_name_en   property_type_en                                                
AL Athbah      Land               843.0   1081.443155  6.221988e+02    53.81   
Abu Hail       Building           183.0   6583.618033  5.938284e+03    89.68   
               Land               211.0   3852.710190  3.902569e+03   322.91   
               Villa             1008.0   5063.234315  4.899030e+03    33.98   
Al Asbaq       Land                17.0  12715.667647  1.148592e+04  2180.07   
...                                 ...           ...           ...      ...   
Zaabeel First  Villa                2.0   2960.075000  3.805578e+02  2690.98   
Zaabeel Second Building             6.0  11948.101667  1.126755e+04   807.29   
               Land                20.0  30283.473000  3.263461e+04    98.46   
               Unit              4054.0  42576.084753  1.275747e+06  3218.60   
Zareeba Duviya Land                27.0  10926.129630  7.669237e+03  2307.87   

                                        25%        50%         75%  \
area_name_en   property_type_en                                      
AL Athbah      Land                672.6200    896.810   1255.5450   
Abu Hail       Building           2906.2650   5318.800   7697.9100   
               Land               1633.9550   2690.980   4683.2350   
               Villa              2522.1900   4602.345   6458.2800   
Al Asbaq       Land               4596.0100   7181.330  17799.6400   
...                                     ...        ...         ...   
Zaabeel First  Villa              2825.5275   2960.075   3094.6225   
Zaabeel Second Building           1950.7250  10795.205  19690.4975   
               Land               3940.8100  20396.700  47179.6950   
               Unit              19281.4475  21910.110  25800.5450   
Zareeba Duviya Land               5995.2800   7250.580  12820.0500   

                                         max  
area_name_en   property_type_en               
AL Athbah      Land                  6405.82  
Abu Hail       Building             40902.44  
               Land                 31342.28  
               Villa               107604.02  
Al Asbaq       Land                 46225.98  
...                                      ...  
Zaabeel First  Villa                 3229.17  
Zaabeel Second Building             27632.75  
               Land                 93284.23  
               Unit              81250000.00  
Zareeba Duviya Land                 29007.24  

[623 rows x 8 columns]

In [ ]:
df.columns


Index(['transaction_id', 'procedure_id', 'trans_group_id', 'trans_group_en',
       'procedure_name_en', 'instance_date', 'property_type_id',
       'property_type_en', 'property_sub_type_id', 'property_sub_type_en',
       'property_usage_en', 'reg_type_id', 'reg_type_en', 'area_id',
       'area_name_en', 'building_name_en', 'project_number', 'project_name_en',
       'master_project_en', 'nearest_landmark_en', 'nearest_metro_en',
       'nearest_mall_en', 'rooms_en', 'has_parking', 'procedure_area',
       'actual_worth', 'meter_sale_price', 'rent_value', 'meter_rent_price',
       'no_of_parties_role_1', 'no_of_parties_role_2', 'no_of_parties_role_3',
       'instance_Year', 'instance_Month', 'instance_Day', 'trans_X_1',
       'trans_X_2', 'Trans_Year', 'trans_X_3'],
      dtype='object')

In [ ]:
def bivariate_analysis(df):
  cat_col = [col for col in df.columns if col.endswith('en')]
  for col in cat_col:
    data_des = df['meter_sale_price'].groupby(df[col],dropna= False).describe()
    sheet_name = f"{col}+values"
    write_to_excel( data_des,'all_bivariate_analysis_against_meter_sale_price.xlsx')

In [ ]:
import pandas as pd
from openpyxl import load_workbook

def write_to_excel(df_dict, file_name='output.xlsx'):
    """
    Writes a dictionary of DataFrames to an Excel file, each DataFrame to a separate sheet.

    Args:
        df_dict: A dictionary where keys are sheet names and values are DataFrames.
        file_name: The name of the Excel file (default: 'output.xlsx').
    """
    try:
        # Try to load the existing workbook
        book = load_workbook(file_name)
        writer = pd.ExcelWriter(file_name, engine='openpyxl')
        writer.book = book

    except FileNotFoundError:
        # If file doesn't exist, create a new workbook and writer
        writer = pd.ExcelWriter(file_name, engine='openpyxl')

    # Write the DataFrames to the Excel file
    for sheet_name, df in df_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save the changes to the file
    writer.save()

In [ ]:
cat_col = [col for col in df.columns if col.endswith('en')]
cat_col

['trans_group_en',
 'procedure_name_en',
 'property_type_en',
 'property_sub_type_en',
 'property_usage_en',
 'reg_type_en',
 'area_name_en',
 'building_name_en',
 'project_name_en',
 'master_project_en',
 'nearest_landmark_en',
 'nearest_metro_en',
 'nearest_mall_en',
 'rooms_en']

In [ ]:
def bivariate_analysis(df):


  property_type = df['meter_sale_price'].groupby(df['property_type_en'],dropna= False).describe()
  property_type.to_excel('property_type.xlsx')

  property_sub_type_en = df['meter_sale_price'].groupby(df['property_sub_type_en'],dropna= False).describe()
  property_sub_type_en.to_excel('property_sub_type_en.xlsx')

  area_name_en = df['meter_sale_price'].groupby(df['area_name_en'],dropna= False).describe()
  area_name_en.to_excel('area_name_en.xlsx')

  project_name_en  = df['meter_sale_price'].groupby(df['project_name_en'],dropna= False).describe()
  project_name_en.to_excel('project_name_en.xlsx')

  area_name = df['meter_sale_price'].groupby(df['area_name_en'],dropna= False).describe()
  area_name.to_excel('area_name.xlsx')

  nearest_landmark_en = df['meter_sale_price'].groupby(df['nearest_landmark_en'],dropna= False).describe()
  nearest_landmark_en.to_excel('nearest_landmark_en.xlsx')

  nearest_metro_en = df['meter_sale_price'].groupby(df['nearest_metro_en'],dropna= False).describe()
  nearest_metro_en.to_excel('nearest_metro_en.xlsx')

  nearest_mall_en = df['meter_sale_price'].groupby(df['nearest_mall_en'],dropna= False).describe()
  nearest_mall_en.to_excel('nearest_mall_en.xlsx')

  procedure_type = df['meter_sale_price'].groupby(df['procedure_name_en'],dropna= False).describe()
  procedure_type.to_excel('procedure_type.xlsx')

  reg_type_en = df['meter_sale_price'].groupby(df['reg_type_en'],dropna= False).describe()
  reg_type_en.to_excel('reg_type_en.xlsx')

  instance_Month = df['meter_sale_price'].groupby(df['instance_Month'],dropna= False).describe()
  instance_Month.to_excel('instance_Month.xlsx')

  instance_Year = df['meter_sale_price'].groupby(df['instance_Year'],dropna= False).describe()
  instance_Year.to_excel('instance_Year.xlsx')

  building_name_en = df['meter_sale_price'].groupby(df['building_name_en'],dropna= False).describe()
  building_name_en.to_excel('building_name_en.xlsx')

  property_usage_en = df['meter_sale_price'].groupby(df['property_usage_en'],dropna= False).describe()
  property_usage_en.to_excel('property_usage_en.xlsx')










In [ ]:
bivariate_analysis(df)

Exception ignored in: <function ZipFile.__del__ at 0x7d45c07ee160>
Traceback (most recent call last):
  File "/usr/lib/python3.11/zipfile.py", line 1895, in __del__
    self.close()
  File "/usr/lib/python3.11/zipfile.py", line 1912, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [ ]:
import pandas as pd

def save_dfs_to_excel(df, file_name='bivariate_analysis.xlsx'):
  """Saves multiple DataFrames to an Excel file with separate sheets.

  Args:
    df: The original DataFrame.
    file_name: The name of the Excel file to save to.
  """
  with pd.ExcelWriter(file_name) as writer:
    df['meter_sale_price'].groupby(df['property_type_en'],dropna= False).describe().to_excel(writer, sheet_name='property_type')
    df['meter_sale_price'].groupby(df['property_sub_type_en'],dropna= False).describe().to_excel(writer, sheet_name='property_sub_type_en')
    df['meter_sale_price'].groupby(df['area_name_en'],dropna= False).describe().to_excel(writer, sheet_name='area_name_en')
    df['meter_sale_price'].groupby(df['project_name_en'],dropna= False).describe().to_excel(writer, sheet_name='project_name_en')
    df['meter_sale_price'].groupby(df['area_name_en'],dropna= False).describe().to_excel(writer, sheet_name='area_name')
    df['meter_sale_price'].groupby(df['nearest_landmark_en'],dropna= False).describe().to_excel(writer, sheet_name='nearest_landmark_en')
    df['meter_sale_price'].groupby(df['nearest_metro_en'],dropna= False).describe().to_excel(writer, sheet_name='nearest_metro_en')
    df['meter_sale_price'].groupby(df['nearest_mall_en'],dropna= False).describe().to_excel(writer, sheet_name='nearest_mall_en')
    df['meter_sale_price'].groupby(df['procedure_name_en'],dropna= False).describe().to_excel(writer, sheet_name='procedure_type')
    df['meter_sale_price'].groupby(df['reg_type_en'],dropna= False).describe().to_excel(writer, sheet_name='reg_type_en')
    df['meter_sale_price'].groupby(df['instance_Month'],dropna= False).describe().to_excel(writer, sheet_name='instance_Month')
    df['meter_sale_price'].groupby(df['instance_Year'],dropna= False).describe().to_excel(writer, sheet_name='instance_Year')
    df['meter_sale_price'].groupby(df['building_name_en'],dropna= False).describe().to_excel(writer, sheet_name='building_name_en')

# Call the function to save the DataFrames
save_dfs_to_excel(df)

KeyboardInterrupt: 

In [ ]:

def save_dfs_to_excel(df, file_name='bivariate_analysis.xlsx'):
  """Saves multiple DataFrames to an Excel file with separate sheets.

  Args:
    df: The original DataFrame.
    file_name: The name of the Excel file to save to.
  """
  with pd.ExcelWriter(file_name) as writer:
    df['meter_sale_price'].groupby(df['trans_group_en'],dropna= False).describe().to_excel(writer, sheet_name='trans_group_en')
    df['meter_sale_price'].groupby(df['rooms_en'],dropna= False).describe().to_excel(writer, sheet_name='rooms_en')
# Call the function to save the DataFrames
save_dfs_to_excel(df)

# Developer data


In [ ]:
d_path = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/DLD_Archive/Developers.csv'
ddf = pd.read_csv(d_path)

In [ ]:
ddf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1888 entries, 0 to 1887
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   participant_id          1888 non-null   int64  
 1   developer_id            1888 non-null   int64  
 2   developer_number        1888 non-null   int64  
 3   developer_name_ar       1888 non-null   object 
 4   developer_name_en       1888 non-null   object 
 5   registration_date       1870 non-null   object 
 6   license_source_id       680 non-null    float64
 7   license_source_ar       680 non-null    object 
 8   license_source_en       680 non-null    object 
 9   license_type_id         1660 non-null   float64
 10  license_type_ar         1639 non-null   object 
 11  license_type_en         1639 non-null   object 
 12  license_number          1804 non-null   object 
 13  license_issue_date      1712 non-null   object 
 14  license_expiry_date     1736 non-null   

In [ ]:
df_cat = [col for col in df.columns if col.endswith('en')]
df_cat

['trans_group_en',
 'procedure_name_en',
 'property_type_en',
 'property_sub_type_en',
 'property_usage_en',
 'reg_type_en',
 'area_name_en',
 'building_name_en',
 'project_name_en',
 'master_project_en',
 'nearest_landmark_en',
 'nearest_metro_en',
 'nearest_mall_en',
 'rooms_en']

In [ ]:
df[df_cat]

,trans_group_en,procedure_name_en,property_type_en,property_sub_type_en,property_usage_en,reg_type_en,area_name_en,building_name_en,project_name_en,master_project_en,nearest_landmark_en,nearest_metro_en,nearest_mall_en,rooms_en
0,Sales,Sell,Villa,NaN,Residential,Existing Properties,Al Bada,NaN,NaN,NaN,Burj Khalifa,Emirates Towers Metro Station,Dubai Mall,NaN
1,Sales,Sell,Villa,NaN,Residential,Existing Properties,Al Bada,NaN,NaN,NaN,Burj Khalifa,Trade Centre Metro Station,Dubai Mall,NaN
2,Sales,Sell,Villa,NaN,Residential,Existing Properties,Al Bada,NaN,NaN,NaN,Burj Khalifa,Emirates Towers Metro Station,Dubai Mall,NaN
3,Sales,Sell,Building,NaN,Residential / Commercial,Existing Properties,Al Karama,NaN,NaN,NaN,Burj Khalifa,ADCB Metro Station,Dubai Mall,NaN
4,Sales,Sell,Building,NaN,Residential,Existing Properties,Al Karama,NaN,NaN,NaN,Burj Khalifa,ADCB Metro Station,Dubai Mall,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148881,Sales,Delayed Sell,Land,NaN,Other,Existing Properties,Al Hebiah Fifth,NaN,DAMAC LAGOONS - MALTA (1),NaN,NaN,NaN,NaN,NaN
1148882,Sales,Sell,Unit,Shop,Commercial,Existing Properties,Al Warsan First,S-05,NaN,International City Phase 1,NaN,Rashidiya Metro Station,City Centre Mirdif,NaN
1148883,Sales,Sell,Unit,Flat,Residential,Existing Properties,Al Warsan First,S-05,NaN,International City Phase 1,NaN,Rashidiya Metro Station,City Centre Mirdif,2 B/R
1148884,Sales,Sell,Unit,Shop,Commercial,Existing Properties,Al Warsan First,S-05,NaN,International City Phase 1,NaN,Rashidiya Metro Station,City Centre Mirdif,NaN


In [ ]:
ddf['developer_name_en']

,developer_name_en
0,MERAAS ESTATES (L.L.C)
1,ZABEEL INVESTMENTS (L.L.C)
2,AL FAJER PROPERTIES (L. L. C)
3,AL MANAL DEVELOPMENT FZCO
4,K M PROPERTIES (L.L.C)
...,...
1883,BABYLONIA LION REAL ESTATE DEVELOPMENT L.L.C
1884,CASA ADUR REAL ESTATE L.L.C
1885,M S B REAL ESTATE DEVELOPMENT L.L.C
1886,CASA PRESTON REAL ESTATE L.L.C


In [ ]:
# Replace 'column_to_split_1' and 'column_to_split_2' with the actual column names
# Replace 'delimiter' with the character used to separate values in the columns (e.g., ',', ';', ' ')
df1['split_column_1'] = df1['column_to_split_1'].str.split('delimiter').str.get(0)  # Get the first element after split
df2['split_column_2'] = df2['column_to_split_2'].str.split('delimiter').str.get(0)  # Get the first element after split

['developer_name_en',
 'license_source_en',
 'license_type_en',
 'legal_status_en']

In [ ]:
dvalues  = ddf['developer_name_en'].unique()
tvalues = df['project_name_en'].unique()

In [ ]:
for i in tvalues:
  print
  if i in dvalues:
    print(i)

The 100


In [ ]:
df1 = df
df2 = ddf

In [ ]:
# Replace 'column_to_split_1' and 'column_to_split_2' with the actual column names
# Replace 'delimiter' with the character used to separate values in the columns (e.g., ',', ';', ' ')
df1['split_column_1'] = df1['project_name_en'].str.split(' ').str.get(0)  # Get the first element after split
df2['split_column_2'] = df2['developer_name_en'].str.split(' ').str.get(0)  # Get the first element after split

In [ ]:
# Using the 'isin' method to find matching values in the split columns
matching_values = df1[df1['split_column_1'].isin(df2['split_column_2'])]['split_column_1'].unique()

# Print the matching values
print(matching_values)

['NEW' 'THE' 'LAKE' 'DUBAI' 'ME' 'PLATINUM' 'AG' 'ELITE' 'GOLD' 'JUMEIRAH'
 'ARMADA' 'TORCH' 'DAMAC' 'CAYAN' 'RUFI' 'HDS' 'MARINA' 'PIER' 'PARK'
 'TFG' 'ESCAN' 'VICTORY' 'SEVEN' 'GRAND' 'BURJ' 'The' 'ZENITH' 'CANAL'
 'UNIESTATE' 'OASIS' 'GLOBAL' 'HAMZA' 'EDEN' 'GOLF' 'GERMAN' 'LIME'
 'CONDOR' 'AL' 'CREEK' 'SKY' 'RP' 'HARBOUR' 'ROYAL' 'SYANN' 'SAMANA' 'LA'
 'ARYENE' 'VINCITORE' 'LINCOLN' 'KENSINGTON' 'Binghatti' 'ACES' 'SEASONS'
 'INDIGO' 'PANTHEON' 'DIAMOND' 'WESTAR' 'PRIME' 'LUCKY' 'EMIRATES'
 'PLAZZO' 'MANHATTAN' 'SHAMAL' 'SIGNATURE' 'MONTE' 'PALACE' 'AURA' 'NORTH'
 'CITY' 'DANA' 'ARTISTIC' 'ONE' 'SIROYA' 'MASAAR' 'BLOOM' 'REEF'
 'EDMONTON' 'GREEN' 'POINT' 'IMPERIAL' 'BUSINESS' 'BONNINGTON' 'SWISS'
 'SABA' 'PALLADIUM' 'MBL' 'ORRA' 'ICON' 'INTERNATIONAL' 'PALAZZO'
 'PROFILE' 'LAGUNA' 'GOLDEN' 'BALQIS' 'ORION' 'MAY' 'SAAS' 'MANAZEL'
 'PALM' 'MYKA' 'AZURE' 'SUN' 'VOLANTE' 'SOBHA' 'CRYSTAL' 'BURLINGTON'
 'AVANTI' 'PARAMOUNT' 'CAPITAL' 'BAY' 'MAG' 'AZIZI' 'DOWNTOWN' 'AVA'
 'ALFURJAN' 'GEM

In [ ]:
import pandas as pd

# Assuming you have already loaded your datasets into df1 and df2

# Split the columns and get values 0 and 1
df1['combined_values'] = df1['project_name_en'].str.split('delimiter').str[:2].str.join(' ')
df2['combined_values'] = df2['developer_name_en'].str.split('delimiter').str[:2].str.join(' ')

# Find matching values
matching_values2 = df1[df1['combined_values'].isin(df2['combined_values'])]['combined_values'].unique()

# Print the matching values
print(matching_values)

['The 100']


In [ ]:
import pandas as pd

# Assuming your datasets are loaded into df1 and df2
# and the list of values is named 'matching_values'

# Create a regular expression pattern from the list of values
pattern = '|'.join(matching_values)

# Filter df1 and df2 based on the pattern
filtered_df1 = df1[df1['project_name_en'].str.contains(pattern, case=False, na=False)]
filtered_df2 = df2[df2['developer_name_en'].str.contains(pattern, case=False, na=False)]

# Now you have filtered_df1 and filtered_df2 containing the records
# that have matching values in their respective columns

In [ ]:
filtered_df1

,Unnamed: 0,transaction_id,procedure_id,trans_group_id,trans_group_en,procedure_name_en,instance_date,property_type_id,property_type_en,property_sub_type_id,...,no_of_parties_role_3,instance_Year,instance_Month,instance_Day,trans_X_1,trans_X_2,Trans_Year,trans_X_3,split_column_1,combined_values
60713,75341,1-102-2024-85657,102,1,Sales,Sell - Pre registration,2024-11-07,3,Unit,60.0,...,0.0,2024.0,11.0,7.0,1,102,2024,85657,The,The 100
60714,75342,1-102-2024-49105,102,1,Sales,Sell - Pre registration,2024-08-19,3,Unit,60.0,...,0.0,2024.0,8.0,19.0,1,102,2024,49105,The,The 100
78458,97319,1-102-2024-88531,102,1,Sales,Sell - Pre registration,2024-11-29,3,Unit,60.0,...,0.0,2024.0,11.0,29.0,1,102,2024,88531,The,The 100
78459,97320,1-102-2024-35252,102,1,Sales,Sell - Pre registration,2024-06-05,3,Unit,60.0,...,0.0,2024.0,6.0,5.0,1,102,2024,35252,The,The 100
96187,119351,1-102-2024-105334,102,1,Sales,Sell - Pre registration,2025-01-02,3,Unit,60.0,...,0.0,2025.0,1.0,2.0,1,102,2024,105334,The,The 100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065576,1321462,1-102-2024-58728,102,1,Sales,Sell - Pre registration,2024-08-19,3,Unit,60.0,...,0.0,2024.0,8.0,19.0,1,102,2024,58728,The,The 100
1101817,1366273,1-102-2024-85638,102,1,Sales,Sell - Pre registration,2024-11-07,3,Unit,60.0,...,0.0,2024.0,11.0,7.0,1,102,2024,85638,The,The 100
1119720,1388566,1-102-2024-86205,102,1,Sales,Sell - Pre registration,2024-11-07,3,Unit,60.0,...,0.0,2024.0,11.0,7.0,1,102,2024,86205,The,The 100
1119721,1388567,1-102-2024-85660,102,1,Sales,Sell - Pre registration,2024-11-07,3,Unit,60.0,...,0.0,2024.0,11.0,7.0,1,102,2024,85660,The,The 100


In [ ]:
import pandas as pd

# Assuming you have already loaded your datasets into df1 and df2

# Split the columns and get the first element after split
df1['split_column_1'] = df1['project_name_en'].str.split('delimiter').str.get(0)
df2['split_column_2'] = df2['developer_name_en'].str.split('delimiter').str.get(0)

# Find matching values
matching_values = df1[df1['split_column_1'].isin(df2['split_column_2'])]['split_column_1'].unique()

# Create a regular expression pattern from the matching values
pattern = '|'.join(matching_values)

# Filter df1 and df2 based on the pattern
filtered_df1 = df1[df1['project_name_en'].str.contains(matching_values, case=False, na=False)]
filtered_df2 = df2[df2['developer_name_en'].str.contains(matching_values, case=False, na=False)]

# Now you have filtered_df1 and filtered_df2 containing the records
# that have matching values in their respective columns

NameError: name 'df1' is not defined

In [ ]:
filtered_df1.groupby('Trans_Year').count()

,Unnamed: 0,transaction_id,procedure_id,trans_group_id,trans_group_en,procedure_name_en,instance_date,property_type_id,property_type_en,property_sub_type_id,...,no_of_parties_role_2,no_of_parties_role_3,instance_Year,instance_Month,instance_Day,trans_X_1,trans_X_2,trans_X_3,split_column_1,combined_values
Trans_Year,,,,,,,,,,,,,,,,,,,,,
2024,76,76,76,76,76,76,76,76,76,76,...,76,76,76,76,76,76,76,76,76,76
2025,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [ ]:
filtered_df2

,participant_id,developer_id,developer_number,developer_name_ar,developer_name_en,registration_date,license_source_id,license_source_ar,license_source_en,license_type_id,...,license_expiry_date,chamber_of_commerce_no,legal_status,legal_status_ar,legal_status_en,webpage,phone,fax,split_column_2,combined_values
1391,568175479,568097230,1799,ذا 100,The 100,12-12-2023,NaN,NaN,NaN,NaN,...,05-12-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The 100,The 100


# Rent contractors

In [ ]:
r_path = '/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/DLD_Archive/Rent_Contracts.csv'
rdf = pd.read_csv(r_path)

NameError: name 'pd' is not defined

# 22/04/2025



In [ ]:
df.columns

Index(['Unnamed: 0', 'transaction_id', 'procedure_id', 'trans_group_id',
       'trans_group_en', 'procedure_name_en', 'instance_date',
       'property_type_id', 'property_type_en', 'property_sub_type_id',
       'property_sub_type_en', 'property_usage_en', 'reg_type_id',
       'reg_type_en', 'area_id', 'area_name_en', 'building_name_en',
       'project_number', 'project_name_en', 'master_project_en',
       'nearest_landmark_en', 'nearest_metro_en', 'nearest_mall_en',
       'rooms_en', 'has_parking', 'procedure_area', 'actual_worth',
       'meter_sale_price', 'rent_value', 'meter_rent_price',
       'no_of_parties_role_1', 'no_of_parties_role_2', 'no_of_parties_role_3',
       'instance_Year', 'instance_Month', 'instance_Day', 'trans_X_1',
       'trans_X_2', 'Trans_Year', 'trans_X_3'],
      dtype='object')

In [ ]:
import pandas as pd

def save_dfs_to_excel(df, file_name='area_wise.xlsx'):
  """Saves multiple DataFrames to an Excel file with separate sheets.

  Args:
    df: The original DataFrame.
    file_name: The name of the Excel file to save to.
  """
  with pd.ExcelWriter(file_name) as writer:
    # Corrected groupby to use a list of columns
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_landmark_en']],dropna= False).count().to_excel(writer, sheet_name='land_mark_count')
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_metro_en']],dropna= False).count().to_excel(writer, sheet_name='metro_count')
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_mall_en']],dropna= False).count().to_excel(writer, sheet_name='mall_count')
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_landmark_en']],dropna= False).describe().to_excel(writer, sheet_name='land_mark_summary')
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_metro_en']],dropna= False).describe().to_excel(writer, sheet_name='metro_summary')
    df['transaction_id'].groupby([df['area_name_en'],df['nearest_mall_en']],dropna= False).describe().to_excel(writer, sheet_name='mall_summary')
    #df['transaction_id'].groupby(df['area_name_en'],df['property_usage_en'],dropnadropna= False).describe().to_excel(writer, sheet_name='property_usage_count
# Call the function to save the DataFrames
save_dfs_to_excel(df)

# 22/04/2025

Web scrapping


In [ ]:
!pip install beautifulsoup4 requests lxml  # Install necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_wikipedia_tables(url):
  """Scrapes tables with labels from a Wikipedia page.

  Args:
    url: The URL of the Wikipedia page.

  Returns:
    A dictionary where keys are table labels and values are Pandas DataFrames.
  """

  response = requests.get(url)
  soup = BeautifulSoup(response.content, "lxml")

  tables = {}
  for table in soup.find_all("table", class_="wikitable"):
    # Get table label (caption)
    caption = table.find("caption")
    if caption:
      label = caption.text.strip()
    else:
      label = "Table"  # Default label if no caption

    # Read table data into Pandas DataFrame
    df = pd.read_html(str(table))[0]
    tables[label] = df

  return tables

# Scrape tables from the provided URL
url = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"
tables = scrape_wikipedia_tables(url)

# Access and process the scraped tables
for label, df in tables.items():
  print(f"Table: {label}")
  # To see the output, run the code.
  # You can further process the DataFrames (df) as needed

Table: Table


<ipython-input-8-65e9fcd30353>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-8-65e9fcd30353>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-8-65e9fcd30353>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-8-65e9fcd30353>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-8-65e9fcd30353>:30: FutureWar

In [ ]:
import pandas as pd

# Assuming 'tables' is your dictionary of DataFrames

# Concatenate the DataFrames in the dictionary
all_dfs = pd.concat(tables.values(), ignore_index=True)

# Create a new column to store the table labels (keys from the dictionary)
all_dfs['Table_Label'] = [k for k, v in tables.items() for _ in range(len(v))]

# Optionally, reset the index
all_dfs = all_dfs.reset_index(drop=True)

# Now you have a single DataFrame containing data from all tables
# with a new column indicating the original table label

In [ ]:
all_dfs

,Community Code,Community Name,Arabic Name,Area (km2),Population (2021),Table_Label
0,911,Madinat Hind 1,مدينة هند 1,14.1,8,Table
1,912,Madinat Hind 2,مدينة هند 2,8.5,2,Table
2,913,Madinat Hind 3,مدينة هند 3,15.3,3623,Table
3,914,Madinat Hind 4,مدينة هند 4,60.4,3136,Table
4,915,Al Yufrah 1,اليفره 1,17.5,403,Table
5,917,Al Marmoom,المرموم,27.3,141,Table
6,919,Al Yufrah 2,اليفره 2,11.8,3,Table
7,921,Al Yalayis 1,الياليس 1,17.8,1679,Table
8,922,Al Yalayis 2,الياليس 2,14.3,3685,Table
9,923,Al Yalayis 3,الياليس 3,12.3,4,Table


In [ ]:
!pip install beautifulsoup4 requests lxml  # Install necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_wikipedia_tables(url):
  """Scrapes tables with labels from a Wikipedia page.

  Args:
    url: The URL of the Wikipedia page.

  Returns:
    A dictionary where keys are table labels and values are Pandas DataFrames.
  """

  response = requests.get(url)
  soup = BeautifulSoup(response.content, "lxml")

  tables = {}
  for table in soup.find_all("table", class_="wikitable"):
    # Get table label (caption)
    caption = table.find("caption")
    if caption:
      label = caption.text.strip()
    else:
      label = "Table"  # Default label if no caption

    # Read table data into Pandas DataFrame
    df = pd.read_html(str(table))[0]
    tables[label] = df

  return tables

# Scrape tables from the provided URL
url = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"
tables = scrape_wikipedia_tables(url)

# Combine all DataFrames into a single DataFrame
all_dfs = pd.concat(tables.values(), ignore_index=True)

# Create a new column to store the table labels (keys from the dictionary)
all_dfs['Table_Label'] = [k for k, v in tables.items() for _ in range(len(v))]

# Optionally, reset the index
all_dfs = all_dfs.reset_index(drop=True)

# Now you have a single DataFrame containing data from all tables
# with a new column indicating the original table label
# To see the output, run the code.

<ipython-input-11-340041c3b0f5>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-11-340041c3b0f5>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-11-340041c3b0f5>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-11-340041c3b0f5>:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-11-340041c3b0f5>:30: Futu

In [ ]:
all_dfs

,Community Code,Community Name,Arabic Name,Area (km2),Population (2021),Table_Label
0,911,Madinat Hind 1,مدينة هند 1,14.1,8,Table
1,912,Madinat Hind 2,مدينة هند 2,8.5,2,Table
2,913,Madinat Hind 3,مدينة هند 3,15.3,3623,Table
3,914,Madinat Hind 4,مدينة هند 4,60.4,3136,Table
4,915,Al Yufrah 1,اليفره 1,17.5,403,Table
5,917,Al Marmoom,المرموم,27.3,141,Table
6,919,Al Yufrah 2,اليفره 2,11.8,3,Table
7,921,Al Yalayis 1,الياليس 1,17.8,1679,Table
8,922,Al Yalayis 2,الياليس 2,14.3,3685,Table
9,923,Al Yalayis 3,الياليس 3,12.3,4,Table


In [ ]:
!pip install beautifulsoup4 requests lxml  # Install necessary libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_all_tables(url):
    """Scrapes all tables from a Wikipedia page.

    Args:
        url: The URL of the Wikipedia page.

    Returns:
        A list of Pandas DataFrames, each representing a table.
    """

    response = requests.get(url)
    soup = BeautifulSoup(response.content, "lxml")

    all_tables = []
    for table in soup.find_all("table"):
        df = pd.read_html(str(table))[0]
        all_tables.append(df)

    return all_tables

# Scrape tables from the provided URL
url = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"
all_tables = scrape_all_tables(url)

# Access and process the scraped tables
for i, df in enumerate(all_tables):
    print(f"Table {i + 1}:")
    # To see the output, run the code.
    # You can further process the DataFrames (df) as needed

Table 1:
Table 2:
Table 3:
Table 4:
Table 5:
Table 6:
Table 7:
Table 8:
Table 9:
Table 10:


<ipython-input-13-0fe89a04abdb>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-13-0fe89a04abdb>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-13-0fe89a04abdb>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-13-0fe89a04abdb>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
<ipython-input-13-0fe89a04abdb>:22: Futu

In [ ]:
all_tables

[    Community Code   Community Name    Arabic Name  Area (km2)  \
 0              101    Nakhlat Deira      نخلة ديرة        99.6   
 1              111      Al Corniche       الكورنيش         0.6   
 2              112           Al Ras          الراس         0.3   
 3              113       Al Dhagaya        الضغاية         0.2   
 4              114        Al Buteen         البطين         0.1   
 5              115        Al Sabkha         السبخة         0.1   
 6              116       Ayal Nasir      عيال ناصر         0.2   
 7              117         Al Murar          المرر         0.4   
 8              118             Naif           نايف         0.7   
 9              119         Al Rigga          الرقة         0.7   
 10             121   Corniche Deira    كورنيش ديرة         0.9   
 11             122        Al Baraha        البراحة         1.0   
 12             123       Al Muteena        المطينة         1.1   
 13             124    Al Muraqqabat       المرقبات         1.

In [ ]:
import pandas as pd

# Assuming 'all_tables' is your list of DataFrames
#all_tables.pop()

# Concatenate the DataFrames vertically, handling different columns
all_tables_df = pd.concat(all_tables, ignore_index=True)

# Now you have a single DataFrame 'all_tables_df'

In [ ]:
community_list = all_tables_df[['Community Code','Community Name','Area (km2)']]

In [ ]:
community_list = community_list.rename(columns={
    'Community Code': 'community_code',
    'Community Name': 'community_name',
    'Area (km2)': 'area_km2'
})

community_list.to_excel('community_list.xlsx')

In [ ]:
def sectors(df,col):
  # Use apply to iterate over rows and apply the logic
  df['secctor'] = df[col].apply(lambda x:
                                 1 if 100 <= x < 200 else
                                 2 if 200 <= x < 300 else
                                 3 if 300 <= x < 400 else
                                 4 if 400 <= x < 500 else
                                 5 if 500 <= x < 600 else
                                 6 if 600 <= x < 700 else
                                 7 if 700 <= x < 800 else
                                 8 if 800 <= x < 900 else
                                 9 if 900 <= x < 1000 else
                                 None  # Handle values outside the ranges
                                )
  return df


In [ ]:
community_code = 'community_code'
sectors(community_list,community_code)

,community_code,community_name,area_km2,secctor
0,101.0,Nakhlat Deira,99.6,1.0
1,111.0,Al Corniche,0.6,1.0
2,112.0,Al Ras,0.3,1.0
3,113.0,Al Dhagaya,0.2,1.0
4,114.0,Al Buteen,0.1,1.0
...,...,...,...,...
228,NaN,NaN,NaN,NaN
229,NaN,NaN,NaN,NaN
230,NaN,NaN,NaN,NaN
231,NaN,NaN,NaN,NaN


In [ ]:
def sectors_name(df,col):
  # Use apply to iterate over rows and apply the logic
  df['sector_name'] = df[col].apply(lambda x:
                                 'North_west'if  1 == x  else
                                 'North' if 2 == x else
                                 'West' if 3 == x else
                                 'North_Central' if 4 == x else
                                 'South_west' if 5 == x else
                                 'Central' if 6 == x else
                                 'North_east' if 7 == x else
                                 'East' if 8  == x else
                                 'South' if 9 == x else
                                 None  # Handle values outside the ranges
                                )

In [ ]:
col = 'secctor'
sectors_name(community_list,col)

In [ ]:
def rural_urban(df,col):
  # Use apply to iterate over rows and apply the logic
  df['urban'] = df[col].apply(lambda x:
                                 'Urban' if 1 <= x <=6 else
                                 'Rural' if 7 <= x <=9 else
                                 None  # Handle values outside the ranges
                                )

In [ ]:
col = 'secctor'
rural_urban(community_list,col)

In [ ]:
community_list.to_excel('community_list.xlsx')

In [ ]:
community_list.sample(10)

,community_code,community_name,area_km2,secctor,sector_name,urban
28,226.0,Al Twar First,2.6,2.0,North,Urban
91,356.0,Umm Suqeim First,2.8,3.0,West,Urban
26,216.0,Al Rashidiya,4.8,2.0,North,Urban
210,931.0,Al Lesaily,112.7,9.0,South,Rural
66,316.0,Al Rifa,1.2,3.0,West,Urban
118,416.0,Nad Al Hammar,8.3,4.0,North_Central,Urban
62,312.0,Al Souk Al Kabir,0.9,3.0,West,Urban
153,624.0,Warsan Fourth,7.9,6.0,Central,Urban
167,676.0,Al Hebiah Third,4.3,6.0,Central,Urban
181,736.0,Nazwah,13.1,7.0,North_east,Rural


# Area_matching


In [ ]:
community_list.columns
tdf.columns

Index(['Unnamed: 0', 'transaction_id', 'procedure_id', 'trans_group_id',
       'trans_group_en', 'procedure_name_en', 'instance_date',
       'property_type_id', 'property_type_en', 'property_sub_type_id',
       'property_sub_type_en', 'property_usage_en', 'reg_type_id',
       'reg_type_en', 'area_id', 'area_name_en', 'building_name_en',
       'project_number', 'project_name_en', 'master_project_en',
       'nearest_landmark_en', 'nearest_metro_en', 'nearest_mall_en',
       'rooms_en', 'has_parking', 'procedure_area', 'actual_worth',
       'meter_sale_price', 'rent_value', 'meter_rent_price',
       'no_of_parties_role_1', 'no_of_parties_role_2', 'no_of_parties_role_3',
       'instance_Year', 'instance_Month', 'instance_Day', 'trans_X_1',
       'trans_X_2', 'Trans_Year', 'trans_X_3'],
      dtype='object')

In [ ]:
#from ctypes import c_void_p
c_names = community_list['community_name'].unique()
p_name = tdf['area_name_en'].unique()
c_id = community_list['community_code'].unique()
p_id = tdf['area_id'].unique()

In [ ]:
names_match = [val for val in p_name if val in c_names]
id_match = [id for id in c_id if id in p_id]

In [ ]:
print(id_match)
print(len(id_match))
print(names_match)
print(len(names_match))


[np.float64(231.0), np.float64(232.0), np.float64(233.0), np.float64(234.0), np.float64(241.0), np.float64(242.0), np.float64(243.0), np.float64(244.0), np.float64(245.0), np.float64(246.0), np.float64(247.0), np.float64(248.0), np.float64(251.0), np.float64(252.0), np.float64(264.0), np.float64(266.0), np.float64(267.0), np.float64(268.0), np.float64(271.0), np.float64(281.0), np.float64(282.0), np.float64(284.0), np.float64(302.0), np.float64(303.0), np.float64(304.0), np.float64(311.0), np.float64(312.0), np.float64(313.0), np.float64(314.0), np.float64(315.0), np.float64(316.0), np.float64(317.0), np.float64(318.0), np.float64(319.0), np.float64(322.0), np.float64(323.0), np.float64(325.0), np.float64(326.0), np.float64(332.0), np.float64(333.0), np.float64(334.0), np.float64(335.0), np.float64(336.0), np.float64(337.0), np.float64(342.0), np.float64(343.0), np.float64(345.0), np.float64(352.0), np.float64(354.0), np.float64(356.0), np.float64(357.0), np.float64(358.0), np.float64(

In [ ]:
match_343 = [id for id in c_id if id == 451]
match_343


[]

In [ ]:
import pandas as pd

# Assuming you have 'tdf' and 'community_list' DataFrames

# Create a dictionary mapping community names to sector names
community_sector_mapping1 = dict(zip(community_list['community_name'], community_list['sector_name']))

# Apply the mapping to create the 'sector_name' column in 'tdf'
tdf['sector_name'] = tdf['area_name_en'].map(community_sector_mapping1).fillna('Unknown')  # Fill unmatched values with 'Unknown'

In [ ]:
community_list[['secctor','sector_name']].sample(10)

,secctor,sector_name
140,5.0,South_west
228,NaN,None
123,4.0,North_Central
142,5.0,South_west
147,6.0,Central
82,3.0,West
109,3.0,West
78,3.0,West
195,8.0,East
149,6.0,Central


In [ ]:
import pandas as pd

# Assuming you have 'tdf' and 'community_list' DataFrames

# Create a dictionary mapping community names to sector names
community_sector_mapping = dict(zip(community_list['community_code'], community_list['secctor']))

# Apply the mapping to create the 'sector_name' column in 'tdf'
tdf['sector_id'] = tdf['area_id'].map(community_sector_mapping).fillna('Unknown')  # Fill unmatched values with 'Unknown'

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
tdf[['area_id','area_name_en','sector_name','sector_id']]

In [ ]:
a = tdf.groupby(['sector_name'])[['area_id', 'area_name_en']].count()

In [ ]:
a.to_excel('sector_count.xlsx')

In [ ]:
a

In [ ]:
b = tdf.sort_values(by= ['sector_name'])[['sector_name','area_id', 'area_name_en']]

In [ ]:
b

In [ ]:
community_list[community_list['community_name']=='Al Thanyah Fifth']

# matching values


BDf area and sectors


In [ ]:
import pandas as pd

# Assuming you have 'tdf' and 'community_list' DataFrames

# Create a dictionary mapping community names to sector names
community_sector_mapping = dict(zip(community_list['community_name'], community_list['sector_name']))

# Apply the mapping to create the 'sector_name' column in 'tdf'
bdf['sector_name'] = bdf['area_name_en'].map(community_sector_mapping).fillna('Unknown')  # Fill unmatched values with 'Unknown'

udf area and sectors

In [ ]:
import pandas as pd

# Assuming you have 'tdf' and 'community_list' DataFrames

# Create a dictionary mapping community names to sector names
community_sector_mapping = dict(zip(community_list['community_name'], community_list['sector_name']))

# Apply the mapping to create the 'sector_name' column in 'tdf'
udf['sector_name'] = udf['area_name_en'].map(community_sector_mapping).fillna('Unknown')  # Fill unmatched values with 'Unknown'

In [ ]:
bdf.columns

Index(['property_id', 'area_id', 'zone_id', 'area_name_en', 'land_number',
       'land_sub_number', 'building_number', 'common_area',
       'actual_common_area', 'floors', 'rooms', 'rooms_en', 'car_parks',
       'built_up_area', 'bld_levels', 'shops', 'flats', 'offices',
       'swimming_pools', 'elevators', 'actual_area', 'property_type_id',
       'property_type_en', 'property_sub_type_id', 'property_sub_type_en',
       'parent_property_id', 'creation_date', 'parcel_id', 'is_free_hold',
       'is_lease_hold', 'is_registered', 'pre_registration_number',
       'master_project_id', 'master_project_en', 'project_id',
       'project_name_en', 'land_type_id', 'land_type_en', 'sector_name'],
      dtype='object')

In [ ]:
bdf[['area_name_en','area_id','sector_name']].sample(10)

,area_name_en,area_id,sector_name
38521,Wadi Al Safa 7,463,Central
106714,Al Wasl,364,West
26083,Madinat Hind 4,505,South
51055,Madinat Al Mataar,462,South_west
21253,Al Yelayiss 1,506,Unknown
14281,Um Suqaim Second,304,Unknown
63768,Al Rashidiya,300,North
45238,Al Hebiah Sixth,531,Central
46853,Dubai Investment Park Second,459,South_west
207894,Um Suqaim Third,370,Unknown


In [ ]:
c = [val for val in community_list['community_name'] if val == 'Al Goze Industrial Third']
c

[]

In [ ]:
community_list['community_name'].nunique()

224

In [ ]:
ldf['sector_name'] = ldf['area_name_en'].map(community_sector_mapping).fillna('Unknown')

In [ ]:
removing_columns = [col for col in ldf.columns if col.endswith('_ar')]

In [ ]:
removing_columns

[]

In [ ]:
ldf.drop(columns=removing_columns,inplace=True)

In [ ]:
ldf.sample()

,property_id,area_id,zone_id,area_name_en,land_number,land_sub_number,actual_area,property_type_id,property_type_en,property_sub_type_id,...,pre_registration_number,separated_from,separated_reference,project_id,project_name_en,master_project_id,master_project_en,land_type_id,land_type_en,sector_name
59616,247372404,267,2,Al Raffa,92,9,248.05,1,Land,62.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Commercial,Unknown


In [ ]:
null_land_numbers = ldf[(ldf['land_number'].isnull()) == True]
   # To see the output, run the code.

In [ ]:
null_land_numbers

,property_id,area_id,zone_id,area_name_en,land_number,land_sub_number,actual_area,property_type_id,property_type_en,property_sub_type_id,...,pre_registration_number,separated_from,separated_reference,project_id,project_name_en,master_project_id,master_project_en,land_type_id,land_type_en,sector_name


In [ ]:
print(len(ldf))
(ldf.isnull().sum()/len(ldf))*100

222442


,0
property_id,0.000000
area_id,0.000000
zone_id,0.000000
area_name_en,0.000000
land_number,0.000000
land_sub_number,0.000000
actual_area,0.521035
property_type_id,0.000000
property_type_en,0.000000
property_sub_type_id,16.698285


In [ ]:
#ldf.to_csv('/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/land_data.csv')
#udf.to_csv('/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/unit_data.csv')
#bdf.to_csv('/content/drive/MyDrive/Flipose-Analytics/TruEstates-2/Data/Data_Model/building_data.csv')

# New try

In [ ]:
!pip install fuzzywuzzy[speedup]  # Install for faster processing
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
def get_closest_match(name, choices):
    """Finds the closest match to 'name' in 'choices'."""
    # Convert 'name' to string to handle potential float values
    name = str(name)
    closest_match = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    if closest_match[1] >= 50:  # Adjust threshold as needed
        return closest_match[0]
    else:
        return 'Unknown'
def matching(df):
  # Apply the function and create the mapping outside the function
  community_list['sector_name'] = community_list['community_name'].apply(
    lambda x: get_closest_match(x, df['area_name_en'].unique())
  )

  community_sector_mapping = dict(zip(community_list['community_name'], community_list['sector_name']))
  df['matching_names'] = df['area_name_en'].map(community_sector_mapping).fillna('Unknown')


In [ ]:
#tdf['matching_names'] = tdf['area_name_en'].map(community_sector_mapping).fillna('Unknown')
#bdf['matching_names'] = bdf['area_name_en'].map(community_sector_mapping).fillna('Unknown')
#udf['matching_names'] = udf['area_name_en'].map(community_sector_mapping).fillna('Unknown')

In [ ]:
matching(bdf)

In [ ]:
bdf.columns

Index(['property_id', 'area_id', 'zone_id', 'area_name_en', 'land_number',
       'land_sub_number', 'building_number', 'common_area',
       'actual_common_area', 'floors', 'rooms', 'rooms_en', 'car_parks',
       'built_up_area', 'bld_levels', 'shops', 'flats', 'offices',
       'swimming_pools', 'elevators', 'actual_area', 'property_type_id',
       'property_type_en', 'property_sub_type_id', 'property_sub_type_en',
       'parent_property_id', 'creation_date', 'parcel_id', 'is_free_hold',
       'is_lease_hold', 'is_registered', 'pre_registration_number',
       'master_project_id', 'master_project_en', 'project_id',
       'project_name_en', 'land_type_id', 'land_type_en', 'sector_name',
       'matching_names'],
      dtype='object')

In [ ]:
bdf.groupby(bdf['sector_name'])[['property_id', 'area_id', 'area_name_en','matching_names']].count()

,property_id,area_id,area_name_en,matching_names
sector_name,,,,
Central,59777,60965,60965,60965
East,41,276,276,276
North,4786,16376,16376,16376
North_Central,279,289,289,289
North_east,2,25,25,25
North_west,2029,9675,9675,9675
South,21401,21482,21482,21482
South_west,16450,16733,16733,16733
Unknown,49724,68511,68511,68511


# merging

In [ ]:
tdf.merge(bdf, on='matching_names', how='inner')

In [ ]:
bdf_subset = bdf[['matching_names', 'master_project_en', 'project_id',
       'project_name_en',]]  # Select relevant columns
tdf_subset = tdf[['matching_names', 'building_name_en',
       'project_number', 'project_name_en', 'master_project_en',
       'nearest_landmark_en', 'nearest_metro_en', 'nearest_mall_en',
       'rooms_en', 'has_parking', 'procedure_area', 'actual_worth',
       'meter_sale_price', 'rent_value', 'meter_rent_price']]
merged_df = tdf_subset.merge(bdf_subset, on='matching_names', how='inner')
#Removed the repeated block of code with unexpected indent.

In [ ]:
import pandas as pd

def merge_with_chunks(path1, path2, chunk_size=10000, merge_column='matching_names', how='inner'):
    """Merges two DataFrames using chunks to avoid memory issues.

    Args:
        path1: Path to the CSV file for the first DataFrame.
        path2: Path to the CSV file for the second DataFrame.
        chunk_size: Number of rows to read in each chunk.
        merge_column: The column to merge on.
        how: Type of merge to perform (e.g., 'inner', 'left', 'right').

    Returns:
        The merged DataFrame.
    """

    merged_chunks = []
    for chunk in pd.read_csv(path1, chunksize=chunk_size):
        merged_chunk = chunk.merge(pd.read_csv(path2), on=merge_column  )
        merged_chunks.append(merged_chunk)

    merged_df = pd.concat(merged_chunks)
    return merged_df

# Example usage:
merged_df = merge_with_chunks(path3, path2, chunk_size=10000)
# To see the output, run the code.

<ipython-input-88-7230ecf592e0>:19: DtypeWarning: Columns (6,31,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_chunk = chunk.merge(pd.read_csv(path2), on=merge_column  )


KeyError: 'matching_names'